In [1]:
import sagemaker
import boto3
import os
import wandb
from sagemaker.pytorch import PyTorch

In [2]:
try:
    sagemaker_session = sagemaker.Session()
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_permissions')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name Shivam to get Role path.


In [3]:
from datetime import datetime

bucket = "ai-sagemaker-weights"
base_job_name = "seat-gen-unpaired"
checkpoint_suffix = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
checkpoint_s3_path = f"s3://{bucket}/{base_job_name}-{checkpoint_suffix}"
print(checkpoint_s3_path)

s3://ai-sagemaker-weights/seat-gen-unpaired-2022-06-07-13-25-06


In [4]:
source_dir = os.getcwd()
wandb.sagemaker_auth(path=source_dir)

estimator = PyTorch(entry_point='sage_train.sh',
                    source_dir=source_dir,
                    role=role,
                    py_version='py38',
                    framework_version='1.11.0',
                    base_job_name=base_job_name,
                    checkpoint_s3_uri=checkpoint_s3_path,
#                     input_mode='FastFile',
                    instance_count=1,
                    instance_type='ml.g5.xlarge',
                    use_spot_instances=True,  # Use a spot instance
                    max_run=3*24*60*60,  # Max training time
                    max_wait=3*24*60*60,  # Max training time + spot waiting time seconds
                    hyperparameters={
                        # "encoder": "legacy_seresnet18",
                        "n_epochs": 50,
                        "batch_size": 8,
                        "project": "new_cut",
                        "use_wandb": True,
                        "name": "no_windows/",
                    })

In [ ]:
estimator.fit({'train': 's3://ai-sagemaker-datasets/seat_gen_unpaired/'}, wait=True)